<a href="https://colab.research.google.com/github/gragbag/NestedFunctionSyntaxChecker/blob/main/cs3110_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

class PDA:
  def __init__(self):
    self.stack = ['Z']
    self.state = 'q0'
    self.path = []
    self.terminals = {'int', 'float', 'void', '=', ',', '+', '*', '{', '}', '(', ')', ';', 'return'}
    self.transitions = {
        ('q0', None, 'Z'): [('q1', 'FuncDecl Z'), ('q2', 'Type Assign Z')],

        #q1 FuncDecl
        ('q1', None, 'FuncDecl'): [('q1', 'ReturnType FunctionName ( ParamList ) Block')],

        #q1 ReturnType
        ('q1', 'int', 'ReturnType'): [('q1', None)],
        ('q1', 'float', 'ReturnType'): [('q1', None)],
        ('q1', 'void', 'ReturnType'): [('q1', None)],

        #q2 Type
        ('q2', 'int', 'Type'): [('q7', None)],
        ('q2', 'float', 'Type'): [('q7', None)],

        #q3 ParamList
        ('q1', 'var', 'FunctionName'): [('q3', None)],

        ('q3', '(', '('): [('q3', None)],
        ('q3', ')', ')'): [('q3', None)],
        ('q3', None, 'ParamList'): [('q4', 'Param'), ('q4', 'Param , ParamList'), ('q3', None)],
        ('q3', ',', ','): [('q3', None)],

        #q4 Params
        ('q4', 'int', 'Param'): [('q3', 'var')],
        ('q4', 'float', 'Param'): [('q3', 'var')],
        ('q3', 'var', 'var'): [('q3', None)],

        #q3 Block, q5 StatementList
        ('q3', '{', 'Block'): [('q5', 'StmtList }')],

        ('q5', None, 'StmtList'): [('q6', 'Stmt ;'), ('q6', 'Stmt ; StmtList')],
        ('q5', '}', '}'): [('q0', None)], #When the function declaration ends, go back to q0
        ('q6', ';', ';'): [('q5', None)],
        ('q6', ';', 'Z'): [('q0', 'Z')], #When the assignment finishes, go back to z0
        ('q6', ')', ')'): [('q6', None)],
        ('q6', ',', ','): [('q13', None)],
        ('q6', '+', 'Operator'): [('q9', 'Expr')],
        ('q6', '*', 'Operator'): [('q9', 'Expr')],

        #q6 statements
        ('q6', None, 'Stmt'): [('q7', 'Assign'), ('q8', 'Return')],

        #q7 assign
        ('q7', 'var', 'Assign'): [('q9', '= Expr')],

        #q9 Expr and Operator
        ('q9', '=', '='): [('q9', None)],

        ('q9', 'var', 'Expr'): [('q6', None), ('q9', 'Operator')], #('q13', '( ArgList )'), ('q13', '( ArgList ) Operator')],
        ('q9', 'num', 'Expr'): [('q6', None), ('q9', 'Operator')],
        ('q9', None, 'Expr'): [('q11', 'FuncCall'), ('q11', 'FuncCall Operator')],
        ('q9', ';', ';'): [('q5', None)],
        ('q9', ',', ','): [('q13', None)],

        ('q9', '+', 'Operator'): [('q9', 'Expr')],
        ('q9', '*', 'Operator'): [('q9', 'Expr')],

        #q11 FuncCall, q13 ArgList
        ('q11', 'var', 'FuncCall'): [('q13', '( ArgList )')],
        ('q13', '(', '('): [('q13', None)],
        ('q13', ')', ')'): [('q9', None)],
        ('q13', None, 'ArgList'): [('q9', 'Expr'), ('q9', 'Expr , ArgList'), ('q6', None)],

        #q8 Return
        ('q8', 'return', 'Return'): [('q9', 'Expr')],
    }


  def get_possible_transitions(self, symbol, top_of_stack):
    return self.transitions.get((self.state, symbol, top_of_stack))

  def push_to_stack(self, variables):
    if variables:
      for var in reversed(variables.split()):
          self.stack.append(var)

  def classify_token(self, token):
    if token in self.terminals:
      return token
    elif is_number(token):
      return 'num'
    else:
      return 'var'


  def process_input(self, tokens):
    # Recursive backtracking helper function
    def backtrack(index, step):
      if index == len(tokens):
        # If we've processed all tokens and stack is at the initial state, return True
        return self.stack[-1] == 'Z' and self.state == 'q0'

      token = tokens[index]
      # print(f"Stack: {self.stack}, Input: {token}, State: {self.state}, Top: {self.stack[-1]}")


      top_of_stack = self.stack.pop()
      curr_state = self.state
      curr_stack = self.stack.copy()
      curr_path = self.path.copy()

      #Include as token, which can be a keyword, var, or num
      token_type = self.classify_token(token) #Returns token, num, or var
      possible_transitions = self.get_possible_transitions(token_type, top_of_stack)

      if possible_transitions:

        for next_state, variables in possible_transitions:
          # print("possible: ", possible_transitions)
          # print("Now Trying: ", variables)
          self.path.append((step, self.state, token, top_of_stack, next_state, variables))
          self.state = next_state

          self.push_to_stack(variables)

          if backtrack(index + 1, step + 1):
            return True

          self.state = curr_state
          self.stack = curr_stack.copy()
          self.path.pop()

      #Do not include token
      possible_transitions = self.get_possible_transitions(None, top_of_stack)
      if possible_transitions:
        for next_state, variables in possible_transitions:
          # print("possible from None: ", possible_transitions)
          # print("Now Trying From None: ", variables)
          self.path.append((step, self.state, 'None', top_of_stack, next_state, variables))
          self.state = next_state

          self.push_to_stack(variables)

          if backtrack(index, step + 1):
            return True

          self.state = curr_state
          self.stack = curr_stack.copy()
          self.path.pop()

      return False


    return backtrack(0, 0)

  def display_path(self):
    print("Transition Path:")
    for (step, current_state, input_symbol, stack_top, next_state, stack_op) in self.path:
        print(f"Step {step}: From state {current_state}, input '{input_symbol}', stack top '{stack_top}' -> "
              f"Next state {next_state}, stack push: '{stack_op}'")

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [ ]:
class function_checker:
  def __init__(self, path):
    self.path = path
    self.functions = {} #function name: (returnValue, [parameter types needed], {parameter_name: type})
    self.variables = {} #variable name: type
    self.get_functions()

  def get_functions(self):
    for i in range(len(self.path)):
      move = self.path[i]
      input_symbol = move[2]
      stack_top = move[3]

      if stack_top == 'ReturnType':
        function_name = self.path[i + 1][2] #Get the input symbol of the next element
        parameters, parameter_names = self.get_parameters(i + 4)

        self.functions[function_name] = (input_symbol, parameters, parameter_names)

  def get_parameters(self, index):
    parameters = [] #parameter type
    parameter_names = {} #parameter name: type
    while index < len(self.path) and self.path[index][3] == 'Param':
      parameter_type = self.path[index][2]
      parameter_name = self.path[index + 1][2]
      parameters.append(parameter_type)
      parameter_names[parameter_name] = parameter_type
      self.variables[parameter_name] = parameter_type
      index = index + 4

    return (parameters, parameter_names)

  def check_function_calls(self):
    i = 0
    while i < len(self.path):
      move = self.path[i]
      func_name = move[2]
      stack_top = move[3]

      if stack_top == 'FuncCall':
        if func_name not in self.functions:
          print(f"Undefined function: {func_name}")
          return False

        function = self.functions[func_name]
        required_parameters = function[1]
        j = 0
        i = i + 2

        while i < len(self.path) and self.path[i][3] != ')':
          curr_input = self.path[i][2]
          curr_top = self.path[i][3]
          if curr_top == 'FuncCall':
            return_type = self.get_return_type(curr_input)
            if return_type != required_parameters[j]:
              print(f"Function {func_name} did not receive the correct parameters. {required_parameters[j]} was expected but {curr_input} was received")
              return False

            j = j + 1
            i = self.check_parameters(i)
            if i == -1:
              return False

          elif curr_top == 'Expr' and curr_input != 'None':
            var_type = self.get_variable_type(curr_input)
            if var_type == None:
              return False

            if var_type != required_parameters[j]:
              print(f"Function {func_name} did not receive the correct parameters. {required_parameters[j]} was expected but {curr_input} was received")
              return False
            j = j + 1

          i = i + 1

        if j < len(required_parameters):
          print(f"Missing arguments for function '{func_name}'. "
                f"Expected {len(required_parameters)}, got {j}.")
          return False

      i = i + 1
    return True

  def check_parameters(self, index):
    func_name = self.path[index][2]
    if func_name not in self.functions:
        print(f"Undefined function: {func_name}")
        return -1

    function = self.functions[func_name]
    required_parameters = function[1]

    j = 0
    index = index + 2
    while index < len(self.path) and self.path[index][3] != ')':
      curr_top = self.path[index][3]
      curr_input = self.path[index][2]
      next_parameter = required_parameters[j]
      if curr_top == 'Expr' and curr_input != 'None':
        var_type = self.get_variable_type(curr_input)
        if var_type == None:
          return -1

        if var_type != next_parameter:
          print(f"Parameter for function {func_name} is incorrect")
          return -1
        else:
          j += 1
      elif curr_top == 'FuncCall':
        index = self.check_parameters(index + 1)
        if index == -1:
          return -1

      index += 1

    if j < len(required_parameters):
        print(f"Missing arguments for function '{func_name}'. "
              f"Expected {len(required_parameters)}, got {j}.")
        return -1
    return index

  def get_return_type(self, function_name):
    return self.functions[function_name][0]

  def get_variable_type(self, expr):
    if is_number(expr):
      if str.isdigit(expr):
        var_type = 'int'
      else:
        var_type = 'float'
    elif expr != 'None':
      if expr not in self.variables:
        print(f"Undefined variable: {expr}")
        return None
      var_type = self.variables[expr]

    return var_type



In [ ]:
def get_tokens(program):
  token_regex = r'\bint\b|\bfloat\b|\bvoid\b|\breturn\b|\b[a-zA-Z_]\w*\b|\d+[.]?\d*|[,{}();+=*]'
  tokens = re.findall(token_regex, program)
  return tokens


pda = PDA()
# program = """
# int f(int a) {
#   return a + 1;
# }

# int g(int b) {
#   return f(b) * 2;
# }

# int h(int c) {
#   return g(f(c));
# }

# int result = h(5);
# """

program = """
int f(int a, int b) {
  return a + b + 1;
}

int g(int b) {
  return f(b, b) * 2;
}

int h(int c) {
  return g(f(c, 5)) + 3;
}

int result = h(g(h(g(f(g(2), h(5))))));
"""

tokens = get_tokens(program)

print("Program Input: ")
print(program)
print("------------------")

if pda.process_input(tokens):
  print("The program syntax is valid!")
  print("------------------")
  pda.display_path()
  function_checker = function_checker(pda.path)
  print("Correct Use of Function Calls: ", function_checker.check_function_calls())
else:
  print("The program syntax is invalid!")

Program Input: 

int f(int a, int b) {
  return a + b + 1;
}

int g(int b) {
  return f(b, b) * 2;
}

int h(int c) {
  return g(f(c, 5)) + 3;
}

int result = h(g(h(g(f(g(2), h(5))))));

------------------
The program syntax is valid!
------------------
Transition Path:
Step 0: From state q0, input 'None', stack top 'Z' -> Next state q1, stack push: 'FuncDecl Z'
Step 1: From state q1, input 'None', stack top 'FuncDecl' -> Next state q1, stack push: 'ReturnType FunctionName ( ParamList ) Block'
Step 2: From state q1, input 'int', stack top 'ReturnType' -> Next state q1, stack push: 'None'
Step 3: From state q1, input 'f', stack top 'FunctionName' -> Next state q3, stack push: 'None'
Step 4: From state q3, input '(', stack top '(' -> Next state q3, stack push: 'None'
Step 5: From state q3, input 'None', stack top 'ParamList' -> Next state q4, stack push: 'Param , ParamList'
Step 6: From state q4, input 'int', stack top 'Param' -> Next state q3, stack push: 'var'
Step 7: From state q3, inp